# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [2]:
# Import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [18]:
# Importing the dataset from Ironhack's database
driver = 'mysql+pymysql:'
user = 'ironhacker_read'
password = 'ir0nhack3r'
ip = '35.239.232.23'
database = 'pokemon'
connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)
query = """
        SELECT * FROM pokemon_stats
"""
pokemon = pd.read_sql(query, engine)


In [19]:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,,309,39,52,43,60,50,65,1,False


From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [20]:
# your code here
hyp = " Total = HP + Attack + Defense + Sp. Atk + Sp. Def + Speed"
hyp
pokemon["Total verif"] = pokemon["HP"] + pokemon["Attack"] + pokemon["Defense"] + pokemon["Sp. Atk"] + pokemon["Sp. Def"] + pokemon["Speed"]
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Total verif
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,318
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,405
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,525
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False,625
4,4,Charmander,Fire,,309,39,52,43,60,50,65,1,False,309


In [21]:
print(pokemon["Total verif"].equals(pokemon["Total"]))

True


## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [57]:
# your code here
a = pokemon["Type 1"].unique()
b = pokemon["Type 2"].unique()
arr = np.append(a, b)
type_set = set(arr)
types = pd.DataFrame(columns=type_set)

dic = {'Dark': 0, 'Electric': 1, 'Water':2, 'Fighting':3, 'Dragon':4, 'Steel':5, 'Psychic':6, 'Fire':7, 'Ground':8, 'Grass':9, 'Poison':10, 'Flying':11, 'Normal':12, 'Rock':13, 'Bug':14, 'Ghost':15, 'Ice':16, 'Fairy':17, '':18}

for i in range(pokemon.shape[0]):
    lst = np.zeros(19)
    lst[dic[pokemon.loc[i]["Type 1"]]] = 1
    lst[dic[pokemon.loc[i]["Type 2"]]] = 1
    types.loc[i] = lst
    
types = types.drop([''], axis=1)
types.head()

,Dark,Electric,Water,Fighting,Dragon,Steel,Psychic,Fire,Ground,Grass,Poison,Flying,Normal,Rock,Bug,Ghost,Ice,Fairy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [62]:
# your code here
cor = types.merge(pokemon['Total'], right_index=True, left_index=True)
cor = cor.corr()
cor

,Dark,Electric,Water,Fighting,Dragon,Steel,Psychic,Fire,Ground,Grass,Poison,Flying,Normal,Rock,Bug,Ghost,Ice,Fairy,Total
Dark,1.000000,-0.069109,-0.068775,-0.024000,-0.001346,-0.091928,-0.057104,-0.059420,-0.078818,-0.074838,-0.020407,-0.067737,-0.072188,-0.045111,-0.041592,-0.033382,-0.035541,0.029711,0.020971
Electric,-0.069109,1.000000,-0.073778,-0.083285,-0.096131,-0.099639,-0.114850,-0.006844,-0.126894,-0.073986,-0.092030,-0.154994,0.011447,-0.123983,-0.077314,0.000242,-0.067703,-0.012327,-0.021665
Water,-0.068775,-0.073778,1.000000,-0.068775,-0.005161,0.000596,0.051130,-0.080531,-0.051171,-0.039613,-0.086119,-0.071687,-0.055090,-0.048645,-0.065792,-0.059483,-0.061108,-0.056046,0.077786
Fighting,-0.024000,-0.083285,-0.068775,1.000000,-0.044417,-0.026557,-0.038069,0.052430,-0.078818,-0.055525,0.026237,-0.098702,-0.032360,-0.081200,-0.019410,0.015173,-0.035541,-0.135638,0.229705
Dragon,-0.001346,-0.096131,-0.005161,-0.044417,1.000000,0.024540,-0.056108,-0.020771,-0.061537,-0.074036,-0.081400,-0.097645,0.049202,0.047178,0.026479,-0.057042,0.013155,-0.194483,0.109703
Steel,-0.091928,-0.099639,0.000596,-0.026557,0.024540,1.000000,-0.061243,-0.079079,-0.094009,-0.103195,-0.063872,-0.112381,-0.069031,-0.111968,-0.070947,-0.042311,0.027227,-0.042948,0.124688
Psychic,-0.057104,-0.114850,0.051130,-0.038069,-0.056108,-0.061243,1.000000,-0.022621,-0.108248,-0.085471,-0.028855,-0.085097,-0.064677,-0.060536,-0.013459,-0.065851,-0.067651,-0.026556,0.078726
Fire,-0.059420,-0.006844,-0.080531,0.052430,-0.020771,-0.079079,-0.022621,1.000000,-0.097033,-0.053878,-0.060578,-0.102044,0.072085,-0.063544,-0.035908,-0.003872,-0.069360,-0.174535,0.015060
Ground,-0.078818,-0.126894,-0.051171,-0.078818,-0.061537,-0.094009,-0.108248,-0.097033,1.000000,0.110372,-0.081366,-0.117156,-0.072829,-0.034430,0.075316,-0.027476,-0.048756,-0.099271,-0.052592
Grass,-0.074838,-0.073986,-0.039613,-0.055525,-0.074036,-0.103195,-0.085471,-0.053878,0.110372,1.000000,-0.067950,-0.110800,-0.081036,0.121210,0.008736,-0.064726,-0.066495,-0.139539,-0.090441


# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [ ]:
# your code here